In [427]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

In [428]:
height=450
width=450
path="Resources/1.jpg"
img = cv2.imread(path)
print(img.shape)
img = cv2.resize(img, (width, height))
#cv2.imshow("original",img)
#cv2.waitKey(0)
#cv2.destroyAllWindows()


(436, 376, 3)


In [429]:
def img_preprocessing(img):
    img_gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_blur=cv2.GaussianBlur(img_gray, (5, 5), 3)
    img_threshold=cv2.adaptiveThreshold(img_blur, 255, 1, 1, 11, 2)
    return img_threshold

In [430]:
img_final=img_preprocessing(img)
#cv2.imshow("preprocessed",backtorgb)
cv2.waitKey(0)
cv2.destroyAllWindows()
print(img_final.shape)

(450, 450)


In [431]:

img_boundary=img.copy()
img_bigContour=img.copy()
contours, hierarchy=cv2.findContours(img_final, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(img_boundary, contours, -1, (0, 255, 0), 3)
cv2.imshow("contours",img_boundary)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [432]:
def find_largest_contour(contours):
    largest=np.array([])
    max_area=0
    for i in contours:
        area=cv2.contourArea(i)
        if area>50:
            perimeter=cv2.arcLength(i, True)
            vertices=cv2.approxPolyDP(i, 0.02*perimeter, True)
            if area>max_area and len(vertices)==4:
                largest=vertices
                max_area=area
    return largest, max_area

def sort_vertices(largest):
    largest=largest.reshape((4, 2))
    newarr=np.zeros((4, 1, 2), dtype=np.int32)
    add=largest.sum(1)
    newarr[0]=largest[np.argmin(add)]
    newarr[3]=largest[np.argmax(add)]
    diff=np.diff(largest, axis=1)
    newarr[1]=largest[np.argmin(diff)]
    newarr[2]=largest[np.argmax(diff)]
    return newarr

largest, max_area=find_largest_contour(contours)
largest1=sort_vertices(largest)
print(largest1)
cv2.drawContours(img_bigContour, largest1, -1, (0, 255, 0), 25)
#cv2.imshow("contours",img_bigContour)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

boundary_points=np.float32(largest1)
virtual_points=np.float32([[0, 0],[width, 0], [0, height],[width, height]])

[[[101 164]]

 [[396 163]]

 [[101 418]]

 [[396 418]]]


In [433]:
matrix=cv2.getPerspectiveTransform(boundary_points, virtual_points)
img_warp=cv2.warpPerspective(img, matrix, (width, height))
img_warp=cv2.cvtColor(img_warp, cv2.COLOR_BGR2GRAY)
cv2.imshow("warp",img_warp)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [434]:
def split(img):
    rows=np.vsplit(img, 9)
    boxes=[]
    for r in rows:
        col=np.hsplit(r, 9)
        for c in col:
            boxes.append(c)
    return boxes 

boxes=split(img_warp)
#print(len(boxes))
#cv2.imshow("box", boxes[0])
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [435]:
model = load_model('final_model.h5')

In [436]:
result=[]
for image in boxes:
    img_pred=np.asarray(image)
    img_pred = img_pred[4:img_pred.shape[0] - 4, 4:img_pred.shape[1] -4]
    img_pred=cv2.resize(img_pred, (32, 32))
    img_pred=img_pred/255
    img_pred=img_pred.reshape(1, 32, 32, 1)
    prediction=model.predict(img_pred)
    predicted_number=model.predict_classes(img_pred)
    prob=np.amax(prediction)
    
    if(prob>0.8):
        result.append(predicted_number[0])
    else:
        result.append(0)

In [437]:
print(result)

[2, 1, 0, 0, 6, 0, 9, 0, 0, 0, 0, 0, 0, 0, 9, 1, 0, 0, 4, 0, 9, 3, 1, 0, 0, 5, 8, 0, 0, 1, 0, 0, 5, 0, 4, 0, 9, 0, 4, 0, 3, 0, 8, 0, 5, 0, 5, 0, 2, 0, 0, 6, 0, 0, 3, 8, 0, 0, 4, 0, 5, 0, 6, 0, 0, 6, 7, 0, 0, 0, 0, 2, 0, 0, 7, 0, 8, 0, 3, 0, 9]


In [438]:
print(img.shape)

(450, 450, 3)


In [439]:
img_blank=np.zeros((height, width, 3), np.uint8)
img_detected=img_blank.copy()

In [440]:
def display_detected(img_detected, result, color = (255, 105, 180)):
    secW = int(img_detected.shape[1]/9)
    secH = int(img_detected.shape[0]/9)
    for x in range (0,9):
        for y in range (0,9):
            if result[(y*9)+x] != 0 :
                 cv2.putText(img_detected, str(result[(y*9)+x]),
                               (x*secW+int(secW/2)-10, int((y+0.8)*secH)), cv2.FONT_HERSHEY_COMPLEX_SMALL,
                            2, color, 2, cv2.LINE_AA)
    return img_detected


In [441]:
img_1=display_detected(img_detected, result, color=(255, 105, 180))
cv2.imshow('detected', img_1)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [442]:
print(img_1)

[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]


In [443]:
result=np.asarray(result)
pos_array=np.where(result>0, 0, 1)
print(pos_array)

[0 0 1 1 0 1 0 1 1 1 1 1 1 1 0 0 1 1 0 1 0 0 0 1 1 0 0 1 1 0 1 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 1 0 1 1 0 0 1 1 0 1 0 1 0 1 1 0 0 1 1 1 1 0 1 1
 0 1 0 1 0 1 0]


In [444]:
import Sudoku
puzzle=np.array_split(result, 9)


try:
    Sudoku.solve(puzzle)
except:
    pass
print(puzzle)
    

[array([2, 1, 5, 4, 6, 8, 9, 3, 7], dtype=int64), array([7, 3, 8, 5, 2, 9, 1, 6, 4], dtype=int64), array([4, 6, 9, 3, 1, 7, 2, 5, 8], dtype=int64), array([6, 2, 1, 8, 9, 5, 7, 4, 3], dtype=int64), array([9, 7, 4, 1, 3, 6, 8, 2, 5], dtype=int64), array([8, 5, 3, 2, 7, 4, 6, 9, 1], dtype=int64), array([3, 8, 2, 9, 4, 1, 5, 7, 6], dtype=int64), array([1, 9, 6, 7, 5, 3, 4, 8, 2], dtype=int64), array([5, 4, 7, 6, 8, 2, 3, 1, 9], dtype=int64)]


In [453]:
arr=[]
for i in puzzle:
    for j in i:
        arr.append(j)
solved=arr*pos_array
print(solved)
img_solved=img_blank.copy()
img_solved=display_detected(img_solved, solved, color=(0, 255, 0))
cv2.imshow('solved', img_solved)
cv2.waitKey(0)
cv2.destroyAllWindows()

[0 0 5 4 0 8 0 3 7 7 3 8 5 2 0 0 6 4 0 6 0 0 0 7 2 0 0 6 2 0 8 9 0 7 0 3 0
 7 0 1 0 6 0 2 0 8 0 3 0 7 4 0 9 1 0 0 2 9 0 1 0 7 0 1 9 0 0 5 3 4 8 0 5 4
 0 6 0 2 0 1 0]


In [454]:
def drawGrid(img):
    secW = int(img.shape[1]/9)
    secH = int(img.shape[0]/9)
    for i in range (0,9):
        pt1 = (0,secH*i)
        pt2 = (img.shape[1],secH*i)
        pt3 = (secW * i, 0)
        pt4 = (secW*i,img.shape[0])
        cv2.line(img, pt1, pt2, (255, 255, 0),2)
        cv2.line(img, pt3, pt4, (255, 255, 0),2)
    return img

In [455]:
virtual_points=np.float32(largest1) # PREPARE POINTS FOR WARP
boundary_points=np.float32([[0, 0],[width, 0], [0, height],[width, height]]) # PREPARE POINTS FOR WARP

In [456]:

matrix=cv2.getPerspectiveTransform(boundary_points, virtual_points)
img_inverse_warp=img.copy()
img_inverse_warp=cv2.warpPerspective(img_solved, matrix, (width, height))
inv_perspective = cv2.addWeighted(img_inverse_warp, 1, img, 0.5, 1)
img_detected=drawGrid(img_detected)
img_solved=drawGrid(img_solved)
cv2.imshow('detected', img_detected)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [457]:
cv2.imshow('solved', img_solved)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [458]:
print(img_inverse_warp.shape, img.shape)

(450, 450, 3) (450, 450, 3)


In [459]:
cv2.imshow('inverse_warp', inv_perspective)
cv2.waitKey(0)
cv2.destroyAllWindows()